In [1]:
from __future__ import annotations

import os 

from torch.utils.data import random_split

from mmpfn.datasets.pad_ufes_20 import PADUFES20Dataset

import os 
import torch 
import numpy as np 
import pandas as pd

from sklearn.metrics import accuracy_score
from mmpfn.models.mmpfn_v2 import MMPFNClassifier
from mmpfn.models.mmpfn_v2.constants import ModelInterfaceConfig
from mmpfn.models.mmpfn_v2.preprocessing import PreprocessorConfig
from mmpfn.scripts_finetune_mm.finetune_tabpfn_main import fine_tune_mmpfn

/home/wall/works/research/MultiModalPFN/mmpfn/models/dino_v2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/home/wall/works/research/MultiModalPFN/mmpfn/models/dino_v2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/home/wall/works/research/MultiModalPFN/mmpfn/models/dino_v2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")
/home/wall/anaconda3/envs/mmpfn2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
# data_path = os.path.join(os.getenv('HOME'), "workspace/works/tabular_image/MultiModalPFN/mmpfn/data/pad_ufes_20")
data_path = os.path.join(os.getenv('HOME'), "works/research/MultiModalPFN/mmpfn/data/pad_ufes_20")
dataset = PADUFES20Dataset(data_path)
# _ = dataset.get_images()
_ = dataset.get_embeddings()

Load embeddings from embeddings/pad_ufes_20/pad_ufes_20_dinov3.pt


In [4]:
accuracy_scores = []
for seed in range(5):
    torch.manual_seed(seed)
    train_len = int(len(dataset) * 0.8)
    test_len = len(dataset) - train_len

    train_dataset, test_dataset = random_split(dataset, [train_len, test_len])

    X_train = train_dataset.dataset.x[train_dataset.indices]
    y_train = train_dataset.dataset.y[train_dataset.indices]
    X_test = test_dataset.dataset.x[test_dataset.indices]
    y_test = test_dataset.dataset.y[test_dataset.indices]
    image_train = train_dataset.dataset.embeddings[train_dataset.indices]
    image_test = test_dataset.dataset.embeddings[test_dataset.indices]
    
    for i in range(X_train.shape[1]):
        col = X_train[:, i]
        col[np.isnan(col)] = np.nanmin(col) - 1
    for i in range(X_test.shape[1]):
        col = X_test[:, i]
        col[np.isnan(col)] = np.nanmin(col) - 1

    torch.cuda.empty_cache()

    save_path_to_fine_tuned_model = "./finetuned_mmpfn_pad_ufes_20.ckpt"
    
    fine_tune_mmpfn(
        # path_to_base_model="auto",
        save_path_to_fine_tuned_model=save_path_to_fine_tuned_model,
        # Finetuning HPs
        time_limit=60,
        finetuning_config={"learning_rate": 0.00001, "batch_size": 1, "max_steps": 100},
        validation_metric="log_loss",
        # Input Data
        X_train=pd.DataFrame(X_train),
        image_train=image_train,
        y_train=pd.Series(y_train),
        categorical_features_index=None,
        device="cuda",  # use "cpu" if you don't have a GPU
        task_type="multiclass",
        # Optional
        show_training_curve=False,  # Shows a final report after finetuning.
        logger_level=0,  # Shows all logs, higher values shows less
        freeze_input=True,  # Freeze the input layers (encoder and y_encoder) during finetuning
        mixer_type='MGM+CQAM', # MGM MGM+CQAM
        mgm_heads=128,
        cqam_heads=12,
    )

    # disables preprocessing at inference time to match fine-tuning
    no_preprocessing_inference_config = ModelInterfaceConfig(
        FINGERPRINT_FEATURE=False,
        PREPROCESS_TRANSFORMS=[PreprocessorConfig(name='none')]
    )

    # Evaluate on Test Data
    model_finetuned = MMPFNClassifier(
        model_path=save_path_to_fine_tuned_model,
        inference_config=no_preprocessing_inference_config, 
        ignore_pretraining_limits=True,
        mixer_type='MGM+CQAM', # MGM MGM+CQAM
        mgm_heads=128,
        cqam_heads=12,
    )

    clf_finetuned = model_finetuned.fit(X_train, image_train, y_train)
    acc_score = accuracy_score(y_test, clf_finetuned.predict(X_test, image_test))
    print("accuracy_score (Finetuned):", acc_score)
    accuracy_scores.append(acc_score)

Fine-tuning Steps:   4%|▍         | 4/100 [00:06<03:35,  2.25s/it, Best Val. Loss=0.521, Best Val. Score=-0.521, Training Loss=0.508, Val. Loss=0.521, Patience=47, Utilization=0, Grad Norm=8.58][2025-09-23 23:31:15,168] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:  17%|█▋        | 17/100 [00:29<02:25,  1.75s/it, Best Val. Loss=0.482, Best Val. Score=-0.482, Training Loss=0.502, Val. Loss=0.49, Patience=35, Utilization=0, Grad Norm=9.08] [2025-09-23 23:31:38,201] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps: 101it [02:44,  1.65s/it, Best Val. Loss=0.471, Best Val. Score=-0.471, Training Loss=0.397, Val. Loss=0.543, Patience=-48, Utilization=0, Grad Norm=8.44]                         
[2025-09-23 23:33:51,993] INFO - Initial Validation Loss: 0.5660938875519983 Best Validation Loss: 0.4705977997870326 Total Steps: 101 Best Step: 36 Total Time Spent: 168.80583572387695


accuracy_score (Finetuned): 0.8652173913043478


Fine-tuning Steps:   2%|▏         | 2/100 [00:02<03:41,  2.26s/it, Best Val. Loss=0.59, Best Val. Score=-0.59, Training Loss=0.524, Val. Loss=0.59, Patience=49, Utilization=0, Grad Norm=6.91][2025-09-23 23:34:00,366] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:  22%|██▏       | 22/100 [00:38<02:33,  1.97s/it, Best Val. Loss=0.521, Best Val. Score=-0.521, Training Loss=0.468, Val. Loss=0.521, Patience=30, Utilization=0, Grad Norm=9.84][2025-09-23 23:34:36,793] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps: 101it [03:03,  1.83s/it, Best Val. Loss=0.461, Best Val. Score=-0.461, Training Loss=0.395, Val. Loss=0.461, Patience=-48, Utilization=0, Grad Norm=8.14]                         
[2025-09-23 23:37:00,544] INFO - Initial Validation Loss: 0.5935690765318904 Best Validation Loss: 0.46058942037713024 Total Steps: 101 Best Step: 100 Total Time Spent: 185.22291922569275


accuracy_score (Finetuned): 0.8652173913043478


Fine-tuning Steps:   3%|▎         | 3/100 [00:03<02:40,  1.66s/it, Best Val. Loss=0.604, Best Val. Score=-0.604, Training Loss=0.43, Val. Loss=0.613, Patience=48, Utilization=0, Grad Norm=9.77][2025-09-23 23:37:10,131] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:  14%|█▍        | 14/100 [00:25<02:46,  1.93s/it, Best Val. Loss=0.538, Best Val. Score=-0.538, Training Loss=0.452, Val. Loss=0.538, Patience=38, Utilization=0, Grad Norm=7.21][2025-09-23 23:37:32,189] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps: 101it [02:45,  1.66s/it, Best Val. Loss=0.503, Best Val. Score=-0.503, Training Loss=0.255, Val. Loss=0.503, Patience=-48, Utilization=0, Grad Norm=5.69]                         
[2025-09-23 23:39:51,566] INFO - Initial Validation Loss: 0.6042756883815655 Best Validation Loss: 0.503314944787894 Total Steps: 101 Best Step: 99 Total Time Spent: 167.3140733242035


accuracy_score (Finetuned): 0.8478260869565217


Fine-tuning Steps:   2%|▏         | 2/100 [00:02<03:16,  2.01s/it, Best Val. Loss=0.524, Best Val. Score=-0.524, Training Loss=0.637, Val. Loss=0.524, Patience=49, Utilization=0, Grad Norm=7.39][2025-09-23 23:39:59,645] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:  13%|█▎        | 13/100 [00:21<02:54,  2.00s/it, Best Val. Loss=0.511, Best Val. Score=-0.511, Training Loss=0.53, Val. Loss=0.511, Patience=39, Utilization=0, Grad Norm=8.06] [2025-09-23 23:40:18,795] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps: 101it [02:45,  1.66s/it, Best Val. Loss=0.455, Best Val. Score=-0.455, Training Loss=0.379, Val. Loss=0.456, Patience=-48, Utilization=0, Grad Norm=6.48]                         
[2025-09-23 23:42:42,553] INFO - Initial Validation Loss: 0.5362367285994355 Best Validation Loss: 0.4547377325446615 Total Steps: 101 Best Step: 90 Total Time Spent: 167.64795780181885


accuracy_score (Finetuned): 0.8543478260869565


Fine-tuning Steps:   2%|▏         | 2/100 [00:02<04:18,  2.64s/it, Best Val. Loss=0.478, Best Val. Score=-0.478, Training Loss=0.549, Val. Loss=0.478, Patience=49, Utilization=0, Grad Norm=7.05][2025-09-23 23:42:51,214] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:   9%|▉         | 9/100 [00:14<02:48,  1.85s/it, Best Val. Loss=0.449, Best Val. Score=-0.449, Training Loss=0.48, Val. Loss=0.449, Patience=43, Utilization=0, Grad Norm=13.9] [2025-09-23 23:43:03,547] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:  26%|██▌       | 26/100 [00:45<02:05,  1.70s/it, Best Val. Loss=0.397, Best Val. Score=-0.397, Training Loss=0.505, Val. Loss=0.42, Patience=27, Utilization=0, Grad Norm=31.4] [2025-09-23 23:43:34,345] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps: 101it [02:50,  1.70s/it, Best Val. Loss=0.377, Best Val. Score=-0.377, Training Loss=0.29, Val. Loss=0.39, Patience=-47, Util

accuracy_score (Finetuned): 0.8478260869565217


In [5]:
# get mean and std of accuracy scores
mean_accuracy = np.mean(accuracy_scores)
std_accuracy = np.std(accuracy_scores)
print("Mean Accuracy:", mean_accuracy)
print("Std Accuracy:", std_accuracy)

Mean Accuracy: 0.8560869565217392
Std Accuracy: 0.007826086956521745


In [6]:
# 1/128/24
# Mean Accuracy: 0.8352173913043478
# Std Accuracy: 0.008630188365773572
# 1/128/16
# Mean Accuracy: 0.8400000000000001
# Std Accuracy: 0.02477116440564191
# 1/64/16
# Mean Accuracy: 0.837391304347826
# Std Accuracy: 0.01623317800225605
# 1/64/16
# Mean Accuracy: 0.8282608695652174
# Std Accuracy: 0.015736487887699012
# 1/64/8
# Mean Accuracy: 0.8273913043478262
# Std Accuracy: 0.016186530842596754
# 2/64/16
# Mean Accuracy: 0.8382608695652174
# Std Accuracy: 0.01893177483224106
# 2/128/16
# Mean Accuracy: 0.8382608695652174
# Std Accuracy: 0.01893177483224106
# Mean Accuracy: 0.8469565217391304
# Std Accuracy: 0.02403526080188738
# 2/128/8
# Mean Accuracy: 0.8482608695652173
# Std Accuracy: 0.015273189626304612
# 2/192/12
# Mean Accuracy: 0.8478260869565217
# Std Accuracy: 0.022717131265414168
# 2/128/12
# Mean Accuracy: 0.8486956521739131
# Std Accuracy: 0.017150506889839996
# 2/64/12
# Mean Accuracy: 0.8404347826086956
# Std Accuracy: 0.018981634741689644
# 1/128/12
# Mean Accuracy: 0.8417391304347825
# Std Accuracy: 0.015998582167803577
# MGM 12
# Mean Accuracy: 0.8330434782608694
# Std Accuracy: 0.01292701630201609
# MGM 8